In [2]:
import pandas as pd

df = pd.read_csv (r'Datos del proyecto\datos_arreglados.csv')
df

,Estacion,Ubicacion,Proyecto,Longitud,Latitud,Radiacion,Fecha,Hora,Horas,Minutos,Valores X
0,Norte,Bucaramanga,CDMB,7.149110,-73.134533,32.00,2010-01-01,17:00,17,0,1700
1,Centro,Bucaramanga,CDMB,7.119262,-73.127154,40.00,2010-01-01,17:00,17,0,1700
2,Centro,Bucaramanga,CDMB,7.119262,-73.127154,42.00,2010-01-02,17:00,17,0,1700
3,Norte,Bucaramanga,CDMB,7.149110,-73.134533,33.00,2010-01-02,17:00,17,0,1700
4,Norte,Bucaramanga,CDMB,7.149110,-73.134533,39.00,2010-01-03,17:00,17,0,1700
...,...,...,...,...,...,...,...,...,...,...,...
43986,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,82.11,2021-01-26,17:59:47,17,59,1759
43987,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,122.08,2021-01-27,17:59:47,17,59,1759
43988,Paralela Bosque,Bucaramanga,Wunderground,7.080521,-73.110565,113.59,2021-01-27,17:39:03,17,39,1739
43989,UPB - Piedecuesta,Piedecuesta,Wunderground,7.038574,-73.069649,106.02,2021-01-28,17:59:47,17,59,1759


In [3]:
df_PB = df[(df.Estacion == 'Paralela Bosque')]
df_UPB = df[(df.Estacion == 'UPB - Piedecuesta')]
df_ciu = df[(df.Estacion == 'Ciudadela')]
df_cen = df[(df.Estacion == 'Centro')]
df_nor = df[(df.Estacion == 'Norte')]
df_flo = df[(df.Estacion == 'Florida')]

df_PB = df_PB.reset_index(drop=True)
df_UPB = df_UPB.reset_index(drop=True)
df_ciu = df_ciu.reset_index(drop=True)
df_cen = df_cen.reset_index(drop=True)
df_nor = df_nor.reset_index(drop=True)
df_flo = df_flo.reset_index(drop=True)

In [4]:
def promedio_mensual(anno,mes,estacion):

    cont = 0
    acum = 0
    for i in range(len(estacion)): 
        fecha_actual = estacion.loc[i,"Fecha"]
        fecha_split = fecha_actual.split('-')
        anno_actual = int(fecha_split[0])
        mes_actual = fecha_split[1]
        if(mes_actual[0] == '0'):
            mes_actual = int(mes_actual[1])
        else:
            mes_actual = int(mes_actual)

        if((mes_actual == mes) and (anno_actual == anno)):
            radiacion_actual = estacion.loc[i,"Radiacion"]
            acum = radiacion_actual + acum
            cont = cont + 1
            
    if(cont == 0):
        return (-1) #si no hay mediciones para el año o el mes requerido, devolverá un -1
    else:
        prom = acum/cont
        return (prom)


In [5]:
def interpolacion_espacial(proms_radiacion,distancias,beta):
    num = 0
    den = 0
    for i in range(len(distancias)):
        if(proms_radiacion[i] != -1): #si encuentra una estación donde no haya medición, no la toma en cuenta.     
            num = num + ((proms_radiacion[i])/(distancias[i]**beta))
            den = den + (1/(distancias[i]**beta))
    #print(proms_radiacion)
    if(den==0):
        return -1 #No hay medición en ninguna estación para esa fecha
    else:
        return(num/den)

In [6]:
ano_req = 2014
mes_req = 3
proms_radiacion = [promedio_mensual(ano_req,mes_req,df_PB),promedio_mensual(ano_req,mes_req,df_UPB),promedio_mensual(ano_req,mes_req,df_ciu),promedio_mensual(ano_req,mes_req,df_cen),promedio_mensual(ano_req,mes_req,df_nor),promedio_mensual(ano_req,mes_req,df_flo)]
distancias = [1.28,7.42,2.03,3.58,6.99,2.28] #deben estar en este orden: PB, UPB, ciu, cen, nor, flo
beta = 2
print(interpolacion_espacial(proms_radiacion,distancias,beta))

0.0


In [7]:
ano_req = 2016
mes_req = 10
proms_radiacion = [promedio_mensual(ano_req,mes_req,df_PB),promedio_mensual(ano_req,mes_req,df_UPB),promedio_mensual(ano_req,mes_req,df_ciu),promedio_mensual(ano_req,mes_req,df_cen),promedio_mensual(ano_req,mes_req,df_nor),promedio_mensual(ano_req,mes_req,df_flo)]
distancias = [0.483,0.807,3.9,5.4,8.78,5.88] #deben estar en este orden: PB, UPB, ciu, cen, nor, flo
beta = 2
print(interpolacion_espacial(proms_radiacion ,distancias,beta))

12.555947805654617


In [11]:
cont = 2010
while cont != 2022:
    x = df["Fecha"].str.startswith(str(cont))
    cont_estaciones = len(df[x].Estacion.unique())
    print(cont,":", cont_estaciones)
    cont += 1
    

2010 : 2
2011 : 2
2012 : 2
2013 : 2
2014 : 4
2015 : 3
2016 : 4
2017 : 4
2018 : 4
2019 : 4
2020 : 2
2021 : 2


# Estimación temporal empírica

In [20]:
import time

dfTimes = {"n":"Time(ns)"}

for i in range(100,len(df_PB),100):
    start = time.perf_counter_ns()
    ano_req = 2018
    mes_req = 9
    proms_radiacion = [promedio_mensual(ano_req,mes_req,df_PB.head(i)),promedio_mensual(ano_req,mes_req,df_UPB.head(i)),promedio_mensual(ano_req,mes_req,df_ciu.head(i)),promedio_mensual(ano_req,mes_req,df_cen.head(i)),promedio_mensual(ano_req,mes_req,df_nor.head(i)),promedio_mensual(ano_req,mes_req,df_flo.head(i))]
    distancias = [0.483,0.807,3.9,5.4,8.78,5.88] #deben estar en este orden: PB, UPB, ciu, cen, nor, flo
    beta = 2
    interpolacion_espacial(proms_radiacion,distancias,beta)
    end = time.perf_counter_ns()
    difference = end-start
    dfTimes[i] = difference
    

dfTimes

{'n': 'Time(ns)',
 100: 9467300,
 200: 16487900,
 300: 14676500,
 400: 18015900,
 500: 25031200,
 600: 30595200,
 700: 32364000,
 800: 38577200,
 900: 45771700,
 1000: 51620100,
 1100: 52027500,
 1200: 59645000,
 1300: 68396700,
 1400: 73855700,
 1500: 78046500,
 1600: 82556900,
 1700: 84526800,
 1800: 86395900,
 1900: 89185000,
 2000: 93101600,
 2100: 100445400,
 2200: 97316100,
 2300: 97400900,
 2400: 99956700,
 2500: 97751000,
 2600: 104983300,
 2700: 119909900,
 2800: 114413700,
 2900: 117548300,
 3000: 123867600,
 3100: 122722800,
 3200: 118494200,
 3300: 124662400,
 3400: 107989500,
 3500: 112057000,
 3600: 137751700,
 3700: 125317900,
 3800: 126406100,
 3900: 123766500,
 4000: 121262900,
 4100: 133220600,
 4200: 127528400,
 4300: 132510800,
 4400: 128508800,
 4500: 139264300,
 4600: 173057700,
 4700: 136758400,
 4800: 136458600,
 4900: 166573900,
 5000: 172216000,
 5100: 291091600,
 5200: 158084100,
 5300: 150083500,
 5400: 153064900,
 5500: 151906500,
 5600: 154295900,
 5700: 1

In [21]:
with open('tiempos3.csv', 'w', encoding='utf-8') as f:
    for key in dfTimes.keys():
        f.write("%s,%s\n"%(key,dfTimes[key]))
